In [ ]:
!pip install datasets
!pip install transformers
!pip install rouge_score
!pip install sentencepiece # need this for pegasus
!pip install evaluate
!pip install transformers[torch]

In [ ]:
import datasets
xsum = datasets.load_dataset("xsum")

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch

model_name = "t5-base"
device = "cuda" if torch.cuda.is_available() else "cpu"
t5_tokenizer = AutoTokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

In [5]:
preds, gts = [], []
counter = 0
prefix = "summarize: "
for idx in range(xsum['test'].num_rows):
  batch = t5_tokenizer(prefix + xsum['test'][idx]['document'], truncation=True, padding="longest", return_tensors="pt").to(device)
  translated = t5_model.generate(**batch)
  tgt_text = t5_tokenizer.batch_decode(translated, skip_special_tokens=True)
  # data_idx = dataset['test'][idx]['id']
  preds.append(tgt_text[0])
  gts.append(xsum['test'][idx]['summary'])
  counter += 1
  if counter % 10 == 0:
    print('done with test', counter)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


done with test 10


In [ ]:
with open('predictions_exp1', 'wb') as f:
  pickle.dump(preds, f)
with open('gts_exp1', 'wb') as f:
  pickle.dump(gts, f)

In [9]:
import evaluate

rouge = evaluate.load("rouge")
result = rouge.compute(predictions=preds, references=gts, use_stemmer=True)
result

{'rouge1': 0.18399279475462316,
 'rouge2': 0.02846386836630739,
 'rougeL': 0.13359333321642863,
 'rougeLsum': 0.13212683541472636}